In [ ]:
## requirements
# allensdk
# scikit-learn > 0.19
# xarray

In [1]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache

/local1/miniconda2/envs/jk/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
boc = BrainObservatoryCache(manifest_file='/local1/data/boc/manifest.json',)

In [3]:
oeid = 541206592

# Initializations:
nwb_dataset = boc.get_ophys_experiment_data(oeid)


In [11]:
import pandas as pd
import xarray as xr

from allensdk.brain_observatory.natural_scenes import NaturalScenes

def get_ns_msr(nwb_dataset):
    ns = NaturalScenes(nwb_dataset)
    mean_sweep_response = ns.mean_sweep_response.copy()
    
    # I don't know what dx is. goodbye!
    mean_sweep_response.drop('dx',axis=1,inplace=True)
    
    # annotate the dataframe with useful indices and columns
    time = pd.Series(
        ns.timestamps[ns.stim_table['start']],
        name='time',
    )
    neurons = pd.Series(
        ns.cell_id,
        name='neuron',
    )
    mean_sweep_response.set_index(time,inplace=True)
    mean_sweep_response.columns = neurons
    
    images = ns.stim_table
    
    return images, mean_sweep_response

In [12]:
y, X = get_ns_msr(nwb_dataset)
print y.head()
print X.head()

   frame  start    end
0     92  16125  16132
1     27  16133  16140
2     52  16140  16147
3     37  16148  16155
4    103  16155  16162
neuron     541510267  541510270  541510307  541510405  588381938  541510410  \
time                                                                          
545.19658   2.750398   3.113332   3.283231   1.035660   2.312769   1.005320   
545.46195   5.472741   4.520462   1.848134   1.509070   3.900594   2.375818   
545.69416   4.938696   1.872071   0.822514  -0.366550   0.590227   0.577107   
545.95953   0.686303  -1.502568  -2.128904  -0.779033  -3.098761   0.632175   
546.19174  -2.763241  -2.317277  -1.518564  -1.131271  -3.716857   0.189942   

neuron     541511183  541510394  588381886  541511196    ...      588381999  \
time                                                     ...                  
545.19658   1.795959   1.302906   1.109441   2.644591    ...       3.245388   
545.46195   0.627758   0.452645  -1.946570  -0.088816    ...       5.05

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [34]:
import numpy as np
def decode(msrx):
    
    # get features and output
    X = msrx.data
    y = msrx['natural_image']
    
    # split training & testing
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y)
    
    # do the classification
    lm = LogisticRegression(
        solver='saga',
        multi_class='ovr',
        penalty='l1',
        n_jobs=-1,
    )
    lm.fit(X_train,y_train)
    return lm.score(X_test,y_test) * len(np.unique(y))

In [35]:
%%timeit -n 1 -r 1
print decode(msrx)

19.9
1 loop, best of 1: 50.1 s per loop
